# Investigate distribution of protein abundances

The overall conclusion of this notebook is that we need to log transform our protein abundances, and that after we do so we can assume it comes from a normal distribution for the purposes of meeting assumptions for parametric tests.

As far as checking the assumption of equal variance between conditions, I'm going to add that to the end of this notebook but haven't done it yet.

### NOTE: Why are the charts blank? Because we use the altair-data-server package to reduce RAM and disk usage by some of the large charts in this notebook. However, a side effect is that you have to run the notebook in order for the data to display on the charts.

In [1]:
import altair as alt
import numpy as np
import oscutils
import scipy
import statsmodels.stats.multitest

alt.data_transformers.enable("data_server") # This prevents our charts from taking up too much RAM and our notebooks from taking up too much disk space; see https://altair-viz.github.io/user_guide/large_datasets.html

DataTransformerRegistry.enable('data_server')

In [2]:
inputs = {}

for source in ["mm", "pd"]:
    df = oscutils.\
    load_protein_table(source, "quant", clean=True).\
    drop(columns="sample").\
    set_index(["sample_type", "sample_condition", "sample_num"])
    
    inputs[source] = df

## Sample counts

See how many samples there are in each condition, to give us a frame of reference.

Throughout this notebook, it is helpful to understand that an individual "condition" within the experiment is the unique combination of both the sample type (e.g. HFL1 single cell) and whether we're looking at healthy or unhealthy. Thus, an example of a single condition would be "HFL1 healthy single cells", as opposed to "HFL1 unhealthy single cells" or "HFL1 healthy boost" or "Unhealthy pseudo-bulk".

In [3]:
def get_sample_counts(inputs_dict):
    
    sources = []
    sample_types = []
    sample_conditions = []
    counts = []
    
    df = inputs_dict["mm"] # Doesn't matter which we use, sample counts are the same for both

    for sample_type in df.index.get_level_values("sample_type").unique():
        for sample_condition in df.index.get_level_values("sample_condition").unique():
            
            # Get the data for just this condition
            df_sel = df.xs((sample_type, sample_condition), level=("sample_type", "sample_condition"), drop_level=False)

            sources.append(source)
            sample_types.append(sample_type)
            sample_conditions.append(sample_condition)
            counts.append(df_sel.shape[0])
    
    return pd.DataFrame({
        "sample_type": sample_types,
        "sample_condition": sample_conditions,
        "count": counts,
    })
                
get_sample_counts(inputs)

,sample_type,sample_condition,count
0,boost,healthy,3
1,boost,unhealthy,3
2,hfl1,healthy,36
3,hfl1,unhealthy,35
4,pbulk,healthy,3
5,pbulk,unhealthy,3


## Histogram of proportion of proteins with different coverages

Make a histogram showing, for each number of samples in each condition, what proportion of proteins are found in that number of samples.

In [4]:
def get_protein_coverages(inputs_dict):
    
    sources = []
    sample_types = []
    sample_conditions = []
    samples_counts = []
    proteins = []
    samples_found_counts = [] 
    
    for source in inputs_dict.keys():
        df = inputs_dict[source]
        for sample_type in df.index.get_level_values("sample_type").unique():
            for sample_condition in df.index.get_level_values("sample_condition").unique():
                
                # Get the data for just this condition
                df_sel = df.xs((sample_type, sample_condition), level=("sample_type", "sample_condition"), drop_level=False)
                
                # Record the coverage for each protein in each condition
                for protein in df_sel.columns:
                    
                    sources.append(source)
                    sample_types.append(sample_type)
                    sample_conditions.append(sample_condition)
                    samples_counts.append(df_sel.shape[0])
                    proteins.append(protein)
                    samples_found_counts.append(df_sel[protein].notna().sum())
    
    coverages_df = pd.DataFrame({
        "source": sources,
        "sample_type": sample_types,
        "sample_condition": sample_conditions,
        "samples_count": samples_counts,
        "protein": proteins,
        "samples_found_count": samples_found_counts,
    })
    
    # Combine some label columns so we can just use a single column for the different groupings we want in the chart
    coverages_df = coverages_df.assign(
        source_type_condition=coverages_df["source"] + "_" + coverages_df["sample_type"] + "_" + coverages_df["sample_condition"],
        source_condition=coverages_df["source"] + "_" + coverages_df["sample_condition"],
    )
    
    return coverages_df
                
coverages = get_protein_coverages(inputs)

In [5]:
# Note: Below we use the altair.vconcat (vertical concatenate) and altair.hconcat (horizontal concatenation) in combination with Python
# list comprehensions to simultaneously display the same type of chart for different experimental conditions. Basically we use the list
# comprehensions to generate the lists of the different charts for each condition, and then use vconcat and hconcat to arrange them how
# we like. The reason that there's a * before each list comprehension is that vconcat and hconcat require charts to be passed as
# individual arguments, rather than in a list; the * character unpacks the elements of each list so that they are passed to the function
# as if they were separate individual arguments (see https://www.geeksforgeeks.org/packing-and-unpacking-arguments-in-python/).
#
# FYI this form is used multiple times in this notebook. If you don't understand all the details it's fine, just focus on reading the charts.

alt.vconcat(*[
    alt.hconcat(*[
        alt.Chart(coverages[
            (coverages["sample_type"] == sample_type) &
            (coverages["source_condition"] == source_condition)
        ]).transform_joinaggregate(
            total="count(*)", # The joinaggregate and calculate transforms allow us to have a y-axis of percentages rather than counts for our histogram. See https://stackoverflow.com/a/56366521
        ).transform_calculate(
            pct="1 / datum.total",
        ).mark_bar().encode(
            x=alt.X(
                "samples_found_count",
                bin=alt.Bin(
                    extent=[0, 37] if sample_type == "hfl1" else [0, 4], # Manually set the range to have a gap at the upper limit of number of samples for each condition, so that the last two bins don't get combined
                    step=1
                ),
                title=[
                    "Number of samples each protein is found in",
                    f"{sample_type}: {source_condition}",
                ],
            ),
            y=alt.Y(
                "sum(pct):Q",
                title="Percentage of all proteins",
                axis=alt.Axis(
                    format="%", # Percentages are easier to read than the default of proportions
                ),
            ),
            color="sample_condition",
        )
        
        for source_condition in coverages["source_condition"].unique()
    ]).resolve_scale(
        x="independent",
        y="shared", # So that all the y axes have the same scale, so it's easy to compare visually across each row
    )
    
    for sample_type in coverages["sample_type"].unique()
])

alt.VConcatChart(...)

## Distributions of abundances for each protein

For a random selection of individual proteins from individual samples, show what the distribution of abundances looks like, just so we can get an idea of the situation. They'll probably be right skewed, since protein abundance data consistently is right skewed.

In [6]:
def individual_distributions_plot(df):
    
    # Randomly choose the proteins we'll show distributions for
    df = df.sample(
        n=25,
        replace=False,
        random_state=0,
        axis=1,
        ignore_index=False,
    )
    
    # Reshape the dataframe so it's easier to plot
    df = pd.melt(
        frame=df,
        var_name="protein",
        value_name="abundance",
        ignore_index=False,
    ).\
    reset_index(drop=False).\
    dropna(
        axis=0,
        how="all",
        subset="abundance",
    )
    
    return alt.vconcat(*[
        alt.hconcat(*[
            alt.hconcat(*[
            
                alt.Chart(df[
                    (df["sample_type"] == sample_type) &
                    (df["sample_condition"] == sample_condition) &
                    (df["protein"] == protein)
                ]).mark_bar().encode(
                    x=alt.X(
                        "abundance:Q",
                        title=f"{sample_condition} - {protein}",
                        bin=True,
                    ),
                    y="count()",
                    color="sample_type:N"
                )

                for sample_condition in df["sample_condition"].unique()
            ])
            for protein in df["protein"].unique()
        ]) 
        for sample_type in df["sample_type"].unique()
    ])

In [7]:
individual_distributions_plot(inputs["mm"])

alt.VConcatChart(...)

In [8]:
individual_distributions_plot(inputs["pd"])

alt.VConcatChart(...)

## Anderson-Darling test for normality

In [9]:
def get_ad_p(ad_res, n):
    
    a2 = ad_res.statistic
    
    # Calculate A*2 by Stephens' method, as given in table 4.7 on page 123 of the following book,
    # available in the BYU library and elsewhere:
    #     D'Agostino, R.B.; Stephens, M.A. (eds.). Goodness-of-Fit Techniques. New York: Marcel Dekker.
    # See also https://en.wikipedia.org/wiki/Anderson%E2%80%93Darling_test
    astar2 = a2 * (1 + 0.75 / n + 2.25 / n ** 2)
    
    # Calculate p for A*2 according to the method in table 4.9 on page 127 of the same book
    # For an explanation of why we only use the upper tail significance level (p) and not the
    # lower tail significance level, see pages 106-107 in the same book. In short, the lower
    # tail significance level corresponds to a superuniform or better than expected fit, which
    # isn't relevant for our use case.
    if astar2 < 0.2:
        p = 1 - np.exp(-13.436 + 101.14 * astar2 - 223.73 * astar2 ** 2)
    elif astar2 < 0.34:
        p = 1 - np.exp( -8.318 + 42.796 * astar2 - 59.938 * astar2 ** 2) 
    elif astar2 < 0.6:
        p =     np.exp( 0.9177 -  4.279 * astar2 -   1.38 * astar2 ** 2)
    elif astar2 < 150:
        p =     np.exp( 1.2937 -  5.709 * astar2 + 0.0186 * astar2 ** 2) 
        
    # Once A*2 > 153, p stops decreasing as A*2 increases, and starts increasing instead. This
    # seems to be a symptom of Stephens' formula for p not being written to handle this range
    # of values for A*2. However, if we get to this point we can be sure that A2 is greater
    # than the critical value for even the most stringent confidence level, so we'll just
    # say p = 0.
    else:
        p = 0
        
    # Verify that the calculated p value agrees with the comparison between A2 and the
    # critical values from scipy.
    #
    # If there is a disagreement and n <= 100, see if it's due to the fact that SciPy only uses
    # the critical value data from table 1.3 of the paper they reference on this; consult table
    # 2 for more information on values that should be used for n <= 100. Using these instead
    # may resolve the issue. Here is the reference, it is available for free online:
    #
    #     Stephens, M A, "EDF Statistics for Goodness of Fit and
    #     Some Comparisons", Journal of the American Statistical
    #     Association, Vol. 69, Issue 347, Sept. 1974, pp 730-737
    #
    # Note that in the SciPy implementation (consult the source code) they calculate the
    # critical values for A2 rather than A2 * (1 + 4 / n - 25 / n **2) as given in the paper
    # by taking the critical values in the paper and dividing them by (1 + 4 / n - 25 / n **2).
    if np.argwhere(ad_res.significance_level > p * 100).shape[0] > 0 and ad_res.critical_values[np.argwhere(ad_res.significance_level > p * 100)[-1][0]] > a2:
        raise ValueError(f"SciPy critical values and calculated p value do not agree. See comment in function source code for ideas.\n\np = {p}\n\nSciPy results:\n\n{ad_res}")
        
    return p

In [10]:
import warnings
warnings.simplefilter("always") # This is so that if we get a warning about a computational error, it isn't silenced after the first occurrence, so we have an idea of how many groups have the issue and thus whether we need to worry about it.

def ad_norm(inputs_dict):
    
    results = {}
    
    for source in inputs_dict.keys():
        df = inputs_dict[source]

        sample_types = []
        sample_conditions = []
        prots = []
        pvals = []

        sample_type = "hfl1" # We won't test for boost and pbulk because they only have n = 3
        for sample_condition in df.index.get_level_values("sample_condition").unique():

            # Get the data for just this condition
            df_sel = df.xs((sample_type, sample_condition), level=("sample_type", "sample_condition"), drop_level=False)
            for prot in df_sel.columns:

                abundances = df_sel[prot].dropna()

                min_count = 15

                if len(abundances) >= min_count:
                    ad_res = scipy.stats.anderson(abundances, dist="norm")
                    p = get_ad_p(ad_res=ad_res, n=len(abundances))
                else:
                    p = np.nan

                sample_types.append(sample_type)
                sample_conditions.append(sample_condition)
                prots.append(prot)
                pvals.append(p)
                    
        raw_pvals = pd.DataFrame({
            "sample_type": sample_types,
            "sample_condition": sample_conditions,
            "protein": prots,
            "p_uncorrected": pvals,
        })

        pvals = raw_pvals[raw_pvals["p_uncorrected"].notna()]

        # Correct the p values
        # Note that we do this independently for each data source (MetaMorpheus or Proteome
        # Discoverer), rather than correcting everything together, because we're analyzing
        # the two tools' outputs independently of eahch other, not together
        reject, pvals_corrected, alphacSidak, alphacBonf = statsmodels.stats.multitest.multipletests(
            pvals=pvals["p_uncorrected"].dropna(),
            alpha=0.05,
            method="fdr_bh",
        )

        pvals = pvals.assign(p_corrected=pvals_corrected) 
        
        results[source] = pvals
        
    sources = []
    sample_types = []
    sample_conditions = []
    counts = []
    props_uncorrected_sig = []
    props_corrected_sig = []

    for source in results.keys():
        df = results[source]
        for sample_type in df["sample_type"].unique():
            for sample_condition in df["sample_condition"].unique():

                df_sel = df[(df["sample_type"] == sample_type) & (df["sample_condition"] == sample_condition)]

                sources.append(source)
                sample_types.append(sample_type)
                sample_conditions.append(sample_condition)

                if df_sel.shape[0] > 0:
                    props_uncorrected_sig.append((df_sel["p_uncorrected"] <= 0.05).sum() / df.shape[0])
                    props_corrected_sig.append((df_sel["p_corrected"] <= 0.05).sum() / df.shape[0])
                else:
                    props_uncorrected_sig.append(np.nan)
                    props_corrected_sig.append(np.nan)

    summary = pd.DataFrame({
        "source": sources,
        "sample_type": sample_types,
        "sample_condition": sample_conditions,
        "Proportion of samples with uncorrected p <= 0.05 (reject null hypothesis of normality)": props_uncorrected_sig,
        "Proportion of samples with corrected p <= 0.05 (reject null hypothesis of normality)": props_corrected_sig,
    })
    
    return summary

ad_norm(inputs)

,source,sample_type,sample_condition,Proportion of samples with uncorrected p <= 0.05 (reject null hypothesis of normality),Proportion of samples with corrected p <= 0.05 (reject null hypothesis of normality)
0,mm,hfl1,healthy,0.395983,0.391815
1,mm,hfl1,unhealthy,0.459265,0.457370
2,pd,hfl1,healthy,0.244981,0.204539
3,pd,hfl1,unhealthy,0.297934,0.266221


## Distribution of normalized abundances across all proteins

Even in our single cell conditions where we have 35-36 samples per condition, although the abundance distributions visually appear right skewed, the relatively small sample size is causing more than half of the samples to still pass the Anderson-Darling test for normality (look at the proportions in the farthest right column in the table output by the previous cell). To be able to do a larger scale overall analysis, we'll mean-normalize the abundances for every protein in each condition, and scale to a range of -1 to 1. That way we can then look at all the protein abundances as one distribution across the entire experiment, which will likely make the right skew more evident. We can then fix this with a log transform.

Note that we also normalize and show distributions for the boost and pseudo-bulk conditions, but the fact that we only have at most 3 data points per protein per condition means that the mean-normalization isn't very effective, because many proteins were only found in 2 of 3 samples, so when we try to mean-normalize, the two abundance data points automatically become 0.5 and -0.5. This is why when you look at the normalized distributions for the boost and pseudo-bulk conditions, you see a bimodal distribution with peaks at 0.5 and -0.5. So we'll just assume that protein abundances in these conditions follow the same trend as in the single cell conditions, and still log transform them.

### Write a function to normalize and combine data across all proteins

In [11]:
def normalize_all(inputs_dict):
    
    normalized = pd.DataFrame()
    
    for source in inputs_dict.keys():
        df = inputs_dict[source]
        for sample_type in df.index.get_level_values("sample_type").unique():
            for sample_condition in df.index.get_level_values("sample_condition").unique():

                # Get the data for just this condition
                df_sel = df.xs((sample_type, sample_condition), level=("sample_type", "sample_condition"), drop_level=False)

                # Calculate the summary statistics we'll need for normalization and scaling
                means = df_sel.mean(axis=0)
                maxes = df_sel.max(axis=0)
                mins = df_sel.min(axis=0)

                # Convert from wide to long table format so we can then join in the summary statistics
                # for each protein in the format and calculate the normalized and scaled abundances for
                # all proteins all at once, instead of having to use another for loop to iterate over
                # each protein and normalize and scale them individually. The logic is the same either
                # way, but this method is more efficient computationally because it puts everything
                # into one big set of columns and operates on those using vectorized pandas/numpy
                # functions that use C under the hood, which is way faster than a Python for loop since
                # C is a compiled language and Python is an interpreted language.
                df_sel = df_sel.melt(var_name="protein", value_name="abundance", ignore_index=False)

                stats = pd.DataFrame({
                    "source": source,
                    "mean": means,
                    "max": maxes,
                    "min": mins,
                })

                df_sel = df_sel.merge(
                    right=stats,
                    left_on="protein", 
                    right_index=True,
                    how="outer",
                )

                # Do our mean-normalization calculation and scale all values to a range of [-1, 1]
                df_sel = df_sel.assign(
                    norm_abundance=(df_sel["abundance"] - df_sel["mean"]) / (df_sel["max"] - df_sel["min"]),
                )
                df_sel = df_sel.reset_index(drop=False)[["source", "sample_type", "protein", "abundance", "norm_abundance"]]

                normalized = pd.concat([normalized, df_sel], axis=0)

    normalized = normalized.dropna(how="any")
    normalized.loc[np.isinf(normalized["norm_abundance"]), "norm_abundance"] = 0
    
    return normalized

### Plot the normalized abundances all together

Lo and behold, it's right skewed!

In [12]:
def overall_distribution_plot(inputs_dict, log2):
    
    normalized = normalize_all(inputs_dict)
    
    if log2:
        normalized = normalized.assign(log2_norm=np.log2(normalized["norm_abundance"] + 1))
    
    chart = alt.vconcat(*[
        alt.Chart(normalized[normalized["source"] == source]).mark_bar().encode(
            x=alt.X(
                "log2_norm" if log2 else "norm_abundance",
                bin=alt.Bin(
                    step=0.05,
                ),
            ),
            y="count()",
            column="sample_type",
        ).properties(
            title=source,
        )
        
        for source in inputs_dict.keys()
    ]).resolve_scale(
        x="shared",
        y="shared",
    )
    
    return chart

In [13]:
overall_distribution_plot(inputs, log2=False)

alt.VConcatChart(...)

### Run the Anderson-Darling test on all the normalized abundances and see if we still pass

In [14]:
def ad_on_normalized(inputs_dict, log2):
    
    normalized = normalize_all(inputs_dict)
   
    if log2:
        normalized = normalized.assign(log2_norm=np.log2(normalized["norm_abundance"] + 1))
        data_colname = "log2_norm"
    else:
        data_colname = "norm_abundance"
    
    sources = []
    sample_types = []
    pvals = []
    
    # We only test the single cell samples because the boost and pseudo-bulk
    # don't normalize well due to small sample sizes--see explanation in text
    # box above
    sample_type = "hfl1"
    
    for source in normalized["source"].unique():
        
        abundances = normalized.loc[
            (normalized["source"] == source) &
            (normalized["sample_type"] == sample_type), data_colname]

        ad_res = scipy.stats.anderson(abundances, dist="norm")
        print(ad_res)
        p = get_ad_p(ad_res=ad_res, n=len(abundances))

        sources.append(source)
        sample_types.append(sample_type)
        pvals.append(p)
        
    return pd.DataFrame({
        "source": sources,
        "sample_type": sample_types,
        "p": pvals,
    })

ad_on_normalized(inputs, log2=False)

AndersonResult(statistic=5581.52445869545, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=1.025145410151576e-18, scale=0.22563686180498116)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
AndersonResult(statistic=1196.9913056344521, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=1.040626272051975e-18, scale=0.2625330095789682)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')


,source,sample_type,p
0,mm,hfl1,0
1,pd,hfl1,0


### log transform and plot the transformed distribution

So since we don't pass, let's see if a log transform fixes it.

In [15]:
overall_distribution_plot(inputs, log2=True)

alt.VConcatChart(...)

### Try the Anderson-Darling test on the log transformed data

In [16]:
ad_on_normalized(inputs, log2=True)

AndersonResult(statistic=3252.9251190708455, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=-0.030585880554048678, scale=0.2861918678443683)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
AndersonResult(statistic=101.48153394483961, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=-0.047861941140412666, scale=0.3708465723851177)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')


,source,sample_type,p
0,mm,hfl1,0.000000e+00
1,pd,hfl1,1.378055e-168


# Conclusion

We will just use the Mann-Whitney U test.